In [6]:
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import json
from llm import LLM, BaseLLM
import os

@dataclass
class ExecutionStep:
    """Enhanced execution step data class"""
    step_id: int
    description: str
    prompt: str
    expected_output: Optional[str] = None
    key_points: Optional[List[str]] = None
    constraints: Optional[List[str]] = None
    dependencies: Optional[List[str]] = None
    result: Optional[str] = None
    check_result: Optional[Dict[str, Any]] = None

class TaskDecomposition:
    def __init__(self, llm: LLM):
        self.llm: BaseLLM = llm
        self.conversation_history: List[Dict[str, str]] = []
        
    def _add_to_history(self, role: str, content: str):
        """Add conversation to history"""
        self.conversation_history.append({"role": role, "content": content})
        
    def generate_plan(self, task_description: str) -> List[ExecutionStep]:
        """Generate detailed task execution plan"""
        planning_prompt = f"""
        Please analyze the following task and create a detailed, step-by-step execution plan.

        Task Description: {task_description}

        Generate a comprehensive execution plan in JSON format following these guidelines:

        1. Step Structure:
        {{
            "steps": [
                {{
                    "step_id": 1,
                    "description": "Detailed step description",
                    "prompt": "Specific execution instructions",
                    "expected_output": "Description of what this step should produce",
                    "key_points": ["Key elements to address", "Important aspects to include"],
                    "constraints": ["Any limitations or requirements to consider"],
                    "dependencies": ["References to previous steps if any"]
                }}
            ]
        }}

        2. Requirements for Each Step:
        - Description should be specific and actionable
        - Prompt should provide clear guidance and context
        - Include all necessary details for execution
        - Consider dependencies on previous steps
        - Specify quality criteria and expectations

        3. Step Planning Considerations:
        - Break down complex tasks into manageable pieces
        - Ensure logical progression between steps
        - Include specific details and examples where relevant
        - Consider edge cases and potential challenges
        - Maintain focus on overall task objectives

        4. Content Guidelines:
        - Be specific rather than generic
        - Include measurable outcomes
        - Provide context for each step
        - Specify any required research or reference materials
        - Include quality checks and validation criteria

        5. Example of Detailed Step:
        {{
            "step_id": 1,
            "description": "Research and analyze the historical context of Victorian England",
            "prompt": "Research and document key aspects of Victorian England (1837-1901): social classes, daily life, technological advancements, and cultural norms. Focus on aspects that directly influence the story's setting. Include specific examples of: clothing styles, transportation methods, social etiquette, and common occupations. Consider how these elements will affect character behavior and plot development.",
            "expected_output": "A comprehensive summary of relevant Victorian era details",
            "key_points": ["Social hierarchy", "Technology level", "Cultural norms", "Daily life aspects"],
            "constraints": ["Must be historically accurate", "Focus on story-relevant details"],
            "dependencies": []
        }}

        Remember:
        - Each step should be self-contained but connected to the overall goal
        - Include enough detail for accurate execution
        - Consider how each step builds towards the final outcome
        - Provide clear success criteria for each step
        - Include specific examples or references where helpful

        Please ensure the generated plan is detailed enough that each step can be executed without requiring additional clarification.
        """
        
        self._add_to_history("user", planning_prompt)
        
        # Generate execution plan
        plan_json = self.llm.generate_json(
            prompt=planning_prompt,
            schema={
                "steps": list
            }
        )
        
        if not plan_json or "steps" not in plan_json:
            raise ValueError("Failed to generate plan")
            
        self._add_to_history("assistant", json.dumps(plan_json, ensure_ascii=False, indent=2))
        
        # Convert to ExecutionStep objects
        steps = []
        for step in plan_json["steps"]:
            steps.append(ExecutionStep(
                step_id=step["step_id"],
                description=step["description"],
                prompt=step["prompt"]
            ))
            
        return steps
        
    def execute_step(self, step: ExecutionStep, previous_results: List[str]) -> str:
        """Execute a single step with enhanced context and guidance"""
        # Build comprehensive context for step execution
        dependencies_context = self._build_dependencies_context(step, previous_results)
        
        context_prompt = f"""
        Task Execution Step {step.step_id}

        Step Description: {step.description}

        Previous Context:
        {dependencies_context}

        Expected Output: {step.expected_output}

        Key Points to Address:
        {self._format_list(step.key_points)}

        Constraints to Consider:
        {self._format_list(step.constraints)}

        Detailed Instructions:
        {step.prompt}

        Requirements for Execution:
        1. Address all key points explicitly
        2. Follow all specified constraints
        3. Maintain alignment with previous steps
        4. Ensure output format matches expectations
        5. Focus on quality and completeness

        Please execute this step and provide a detailed response that meets all requirements.
        """
        
        self._add_to_history("user", context_prompt)
        
        result = self.llm.generate_text(
            prompt=context_prompt,
            max_tokens=2000
        )
        
        self._add_to_history("assistant", result)
        return result

    def check_step_result(self, step: ExecutionStep, result: str, task_description: str) -> Dict[str, Any]:
        """Enhanced result validation with detailed feedback"""
        check_prompt = f"""
        Please perform a comprehensive evaluation of the step execution result.

        Original Task: {task_description}

        Step Information:
        - Description: {step.description}
        - Expected Output: {step.expected_output}
        - Key Points: {self._format_list(step.key_points)}
        - Constraints: {self._format_list(step.constraints)}

        Execution Result:
        {result}

        Please evaluate the result based on the following criteria and return a detailed analysis in JSON format:

        {{
            "passed": boolean,
            "scores": {{
                "completeness": (0-10),  // Did it address all required points?
                "constraints_met": (0-10),  // Were all constraints followed?
                "quality": (0-10),  // Overall quality of the output
                "coherence": (0-10)  // Logical flow and connection with other steps
            }},
            "analysis": {{
                "strengths": ["list", "of", "strengths"],
                "weaknesses": ["list", "of", "weaknesses"],
                "missing_points": ["key points", "not addressed"],
                "violated_constraints": ["constraints", "not met"]
            }},
            "improvement_suggestions": ["specific", "actionable", "suggestions"],
            "overall_feedback": "Detailed explanation of the evaluation"
        }}

        Provide specific examples and references when discussing strengths or weaknesses.
        """
        
        self._add_to_history("user", check_prompt)
        
        check_result = self.llm.generate_json(
            prompt=check_prompt,
            schema={
                "passed": bool,
                "scores": dict,
                "analysis": dict,
                "improvement_suggestions": list,
                "overall_feedback": str
            }
        )
        
        self._add_to_history("assistant", json.dumps(check_result, ensure_ascii=False, indent=2))
        return check_result

    def _build_dependencies_context(self, step: ExecutionStep, previous_results: List[str]) -> str:
        """Build context based on step dependencies"""
        if not step.dependencies or not previous_results:
            return "No dependencies on previous steps."
            
        context = "Relevant context from previous steps:\n\n"
        for dep in step.dependencies:
            step_num = int(dep.split()[-1]) - 1  # Extract step number from dependency
            if 0 <= step_num < len(previous_results):
                context += f"From {dep}:\n{previous_results[step_num]}\n\n"
        return context

    def _format_list(self, items: Optional[List[str]]) -> str:
        """Format list items for prompt display"""
        if not items:
            return "None specified"
        return "\n".join(f"- {item}" for item in items)

    def execute_task(self, task_description: str, enable_checking: bool = True) -> Dict[str, Any]:
        """Execute complete task with enhanced monitoring and control"""
        self.conversation_history = []
        
        # Generate initial plan
        steps = self.generate_plan(task_description)
        
        # Execute steps sequentially
        results = []
        execution_log = []
        
        for step in steps:
            # Execute step
            result = self.execute_step(step, results)
            step.result = result
            
            # Validate result if enabled
            if enable_checking:
                check_result = self.check_step_result(step, result, task_description)
                step.check_result = check_result
                
                # Retry if necessary
                if not check_result["passed"]:
                    retry_prompt = self._generate_retry_prompt(step, check_result)
                    result = self.llm.generate_text(
                        prompt=retry_prompt,
                        max_tokens=2000
                    )
                    step.result = result
                    
                    # Re-check after retry
                    step.check_result = self.check_step_result(step, result, task_description)
            
            results.append(result)
            execution_log.append({
                "step_id": step.step_id,
                "description": step.description,
                "expected_output": step.expected_output,
                "result": step.result,
                "check_result": step.check_result
            })
        
        # Generate final result with enhanced context
        final_result = self._generate_final_result(task_description, results, steps)
        
        return {
            "final_result": final_result,
            "execution_log": execution_log,
            "conversation_history": self.conversation_history
        }

    def _generate_retry_prompt(self, step: ExecutionStep, check_result: Dict[str, Any]) -> str:
        """Generate detailed retry prompt based on validation feedback"""
        return f"""
        The previous execution of step {step.step_id} requires improvement.

        Original Description: {step.description}
        Expected Output: {step.expected_output}

        Previous Result: {step.result}

        Evaluation Feedback:
        - Scores: {json.dumps(check_result['scores'], indent=2)}
        - Missing Points: {', '.join(check_result['analysis']['missing_points'])}
        - Violated Constraints: {', '.join(check_result['analysis']['violated_constraints'])}

        Improvement Requirements:
        {self._format_list(check_result['improvement_suggestions'])}

        Please revise the output addressing all identified issues while maintaining:
        1. Original task objectives
        2. Consistency with previous steps
        3. All specified constraints
        4. Required quality standards

        Provide an improved version that addresses all feedback points.
        """

    def _generate_final_result(self, task_description: str, results: List[str], steps: List[ExecutionStep]) -> str:
        """Generate final result with comprehensive context integration"""
        final_integration_prompt = f"""
        Please create a comprehensive final result integrating all completed steps.

        Original Task Description:
        {task_description}

        Step Results Summary:
        {self._format_steps_summary(steps, results)}

        Requirements for Final Integration:
        1. Ensure perfect alignment with original task requirements
        2. Maintain logical flow and coherence across all components
        3. Address all key points from individual steps
        4. Resolve any inconsistencies between steps
        5. Provide a polished and professional final output

        Create a cohesive final result that successfully achieves all original task objectives while maintaining the quality and detail level of individual steps.
        """

        self._add_to_history("user", final_integration_prompt)
        final_result = self.llm.generate_text(
            prompt=final_integration_prompt,
            max_tokens=3000
        )
        self._add_to_history("assistant", final_result)
        return final_result

    def _format_steps_summary(self, steps: List[ExecutionStep], results: List[str]) -> str:
        """Format detailed summary of all steps and their results"""
        summary = ""
        for step, result in zip(steps, results):
            summary += f"\nStep {step.step_id}: {step.description}\n"
            summary += f"Expected Output: {step.expected_output}\n"
            summary += f"Result: {result}\n"
            if step.check_result:
                summary += f"Quality Scores: {json.dumps(step.check_result.get('scores', {}), indent=2)}\n"
            summary += "-" * 80 + "\n"
        return summary

# Usage example
def example_usage(task:str = None):
    llm = LLM(model_type='claude', verbose=True)
    decomposer = TaskDecomposition(llm)

    # Define task
    task = """
Write a 500-word science fiction story about a time traveler who discovers 
an unexpected consequence of changing the past. Include character development, 
plot twists, and a meaningful resolution.
    """

    # Execute task with enhanced monitoring
    result = decomposer.execute_task(task, enable_checking=True)

    # Access detailed execution log
    for entry in result["execution_log"]:
        print(f"\nStep {entry['step_id']}:")
        print(f"Description: {entry['description']}")
        print(f"Expected Output: {entry['expected_output']}")
        if entry['check_result']:
            print(f"Quality Scores: {entry['check_result']['scores']}")
            print(f"Improvement Suggestions: {entry['check_result']['improvement_suggestions']}")

    # Print final result
    print("\nFinal Result:")
    print(result["final_result"])

In [7]:
example_usage('(使用中文生成和思考)生成一个勇者战胜恶龙的故事，但是主角的同伴背叛他，并且主角有着悲惨的过去。最后主角付出了生命才战胜恶龙，但是成果被恶毒的伙伴侵吞，无人知晓主角的功绩。')

TypeError: Object of type type is not JSON serializable